# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [298]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [299]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [300]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [301]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [302]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [303]:

session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = 
{'class': 'SimpleStrategy', 'replication_factor': 1}
""")

# another strategy is the network topology strategies for multpile data centers
## session.execute("""Create Keyspace if not exists sparkify with replication = {"class":"NetworkTopologyStrategy", "east": 2, "west":3}""")

#### Set Keyspace

In [304]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [305]:

query = 'CREATE TABLE IF NOT EXISTS history (sessionId int, itemInSession int, artist text, song text, song_length float, Primary key (sessionId, itemInSession))'
try:
    session.execute(query)
except Exception as e:
        print(e)
                    

In [306]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO history (artist,itemInSession,song_length,sessionId, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [307]:
query = 'SELECT artist, song, song_length from history where sessionId=338 and itemInSession=4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

column_labels = ["artist", "song title", "song length"]
df = pd.DataFrame(columns=column_labels)
for row in rows:
    row = list(row)
    df.loc[len(df)] = row
df

,artist,song title,song length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [308]:
query = 'CREATE TABLE IF NOT EXISTS songs ( artist text, song text, user text, userid int,iteminsession int, sessionid int, PRIMARY KEY ((userid,sessionid),iteminsession))\
WITH CLUSTERING ORDER BY (iteminsession DESC)'
try:
    session.execute(query)
except Exception as e:
        print(e)


In [309]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (artist,song, user, userid, iteminsession, sessionid)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], '{} {}'.format(line[1], line[4]),int(line[10]), int(line[3]), int(line[8])))
                    

In [310]:
query = 'SELECT artist, song, user from songs where userId=10 and sessionid=182'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
column_labels = ["artist", "song", "user"]
df = pd.DataFrame(columns=column_labels)

for row in rows:
    row = list(row)
    df.loc[len(df)] = row
df
   

,artist,song,user
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz
1,Sebastien Tellier,Kilometer,Sylvie Cruz
2,Three Drives,Greece 2000,Sylvie Cruz
3,Down To The Bone,Keep On Keepin' On,Sylvie Cruz


In [311]:

query = 'CREATE TABLE IF NOT EXISTS users ( firstname text, lastname text, song text, PRIMARY KEY (song))'

try:
    session.execute(query)
except Exception as e:
        print(e)
                    

In [312]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (firstname, lastname, song)"
        query = query + "VALUES(%s, %s, %s)"
        session.execute(query, (line[1], line[4],line[9]))
                    

In [313]:
query = "SELECT firstname, lastname, song from users where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
column_labels = ["firstname", "lastname", "song"]
df = pd.DataFrame(columns=column_labels)

for row in rows:
    row = list(row)
    df.loc[len(df)] = row
df

,firstname,lastname,song
0,Jacqueline,Lynch,All Hands Against His Own


### Drop the tables before closing out the sessions

In [314]:
queries = ["drop table if exists users","drop table if exists songs", "drop table if exists history"]
for query in queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [315]:
session.shutdown()
cluster.shutdown()